### Sentencepiece : NSMC 데이터로 tokenizer 구성해보기
: https://github.com/google/sentencepiece  

내부 단어 분리를 위한 유용한 패키지로 구글의 센텐스피스(Sentencepiece)가 있습니다.

내부 단어 분리 알고리즘을 사용하기 위해서, 데이터에 단어 토큰화를 먼저 진행한 상태여야 한다면 이 단어 분리 알고리즘을 모든 언어에 사용하는 것은 쉽지 않습니다. 영어와 달리 한국어와 같은 언어는 단어 토큰화부터가 쉽지 않기 때문입니다. 그런데, 이런 사전 토큰화 작업(pretokenization)없이 전처리를 하지 않은 데이터(raw data)에 바로 단어 분리 토크나이저를 사용할 수 있다면, 이 토크나이저는 그 어떤 언어에도 적용할 수 있는 토크나이저가 될 것입니다. 센텐스피스는 이 이점을 살려서 구현되었습니다. 센텐스피스는 사전 토큰화 작업없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않습니다.

In [1]:
# !pip install sentencepiece

In [2]:
import pandas as pd
import sentencepiece as spm
import csv

In [3]:
# 데이터 읽어오기 : ratings_train.txt
naver_df = pd.read_table('./data/ratings_train.txt')
print(len(naver_df))
naver_df[:5]

150000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
# Null 값 제거 / 확인
naver_df = naver_df.dropna(how='any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [5]:
print(len(naver_df))

149995


In [6]:
# 최종적으로 전처리된 텍스트를 'naver_review.txt'에 저장 (문장 \n 구분)
with open('./data/naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))
f.close() 

In [7]:
# 학습하기: 2개 파일 생성 .vocab (subword), .model
# byte-pair-encoding
spm.SentencePieceTrainer.Train('--input=./data/naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [8]:
# vocab 파일 확인해보기
vocab_list = pd.read_csv('./data/naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,ᄏᄏ,-6


In [9]:
vocab_list.sample(10)

,0,1
2083,▁라는,-2080
418,▁짜증,-415
560,장면,-557
3921,멘,-3918
3416,공,-3413
4728,펏,-4725
3071,▁사랑스러운,-3068
4942,女,-4939
2567,▁다만,-2564
4401,뻥,-4398


In [10]:
len(vocab_list)

5000

In [11]:
# 모델 로드
sp = spm.SentencePieceProcessor()
vocab_file = "./data/naver.model"
sp.load(vocab_file)

True

In [12]:
# 모델을 통해 tokenizing
lines = [
  "아 정말 피곤하다",
  "오늘은 집에 가고 싶다 ㅋㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line)) # encoding token
  print(sp.encode_as_ids(line)) # encoding idx
  print()

아 정말 피곤하다
['▁아', '▁정말', '▁피', '곤', '하다']
[7, 43, 493, 3852, 79]

오늘은 집에 가고 싶다 ㅋㅋㅋ
['▁오늘', '은', '▁집', '에', '▁가', '고', '▁싶다', '▁ᄏᄏᄏ']
[960, 3310, 467, 3301, 46, 3293, 763, 373]



In [13]:
# 모델로 voca size 확인하기
sp.GetPieceSize()

5000

In [14]:
# id to subword
sp.IdToPiece(7)

'▁아'

In [15]:
# subword to id
sp.PieceToId('▁아')

7

In [16]:
# ids to subwords
sp.DecodeIds([7, 43, 493, 3852, 79])

'아 정말 피곤하다'

In [17]:
# subwords to 원형
sp.DecodePieces(['▁오늘', '은', '▁집', '에', '▁가', '고', '▁싶다', '▁ᄏᄏᄏ'])

'오늘은 집에 가고 싶다 ᄏᄏᄏ'

In [19]:
# encode 메소드 중 out_type 인자 활용: subwords, ids로 변환
print(sp.encode('진짜 끔찍하다 끔찍해', out_type=str))
print(sp.encode('진짜 끔찍하다 끔찍해', out_type=int))

['▁진짜', '▁끔', '찍', '하다', '▁끔', '찍', '해']
[54, 3122, 3683, 79, 3122, 3683, 3336]
